In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
test = pd.read_csv("2019.02.20 Corporate SDG Commitments.csv")

In [3]:
#Coding NaN as SDG 0
test.iloc[:,1:2] = test.iloc[:,1:2].fillna(0)

In [4]:
############################
## Wide to Narrow Helpers ##
############################

#splitInt takes a string s of comma deliniated items and returns an array of those items cast as integers. 
#Ignores whitespace.
def splitInt(s):
    s = str(s)
    return np.array(list(map(int, re.split(r" ?, ?", s))))

#splitStr takes a string s of comma deliniated items and returns an array of those items cast as strings. 
#Ignores whitespace.

def splitStr(s):
    s = str(s)
    return np.array(list(map(str, re.split(r" *[,;] *", s))))

#repRow takes a series ser that represents a dataframe row, a column index col, and a splitter function f.
#Reurns that row repeated, but with column col replaced with one of the split elements from applying f to col.
def repRow(ser, col, f):
    splitted = f(ser[col])
    serDf = pd.DataFrame(ser).transpose()
    retDf = pd.DataFrame(pd.np.repeat(serDf.values, len(splitted), axis=0), columns = serDf.columns)
    retDf.iloc[:,col] = splitted
    return retDf

In [5]:
cleanTest = pd.DataFrame([])
testLen = len(test.iloc[:, 0])
for row in range(testLen):
    cleanTest = cleanTest.append(repRow(test.iloc[row], 1, splitInt))
cleanTest = cleanTest.reset_index().iloc[:, 1:11]
cleanTest

,Corporation,SDG(s),Source Name,Source,Source Type,Source Date,Source Author,Explicit or Implicit,Notes,Relevant Country
0,Adobe,4,Adobe Alignment to UN Sustainable Development ...,https://www.adobe.com/content/dam/acom/en/corp...,PDF,8/1/2018,NaN,Explicit,Adobe is a powerhouse for SDGs. but not for 3 ...,U.S.
1,Adobe,5,Adobe Alignment to UN Sustainable Development ...,https://www.adobe.com/content/dam/acom/en/corp...,PDF,8/1/2018,NaN,Explicit,Adobe is a powerhouse for SDGs. but not for 3 ...,U.S.
2,Adobe,8,Adobe Alignment to UN Sustainable Development ...,https://www.adobe.com/content/dam/acom/en/corp...,PDF,8/1/2018,NaN,Explicit,Adobe is a powerhouse for SDGs. but not for 3 ...,U.S.
3,Adobe,12,Adobe Alignment to UN Sustainable Development ...,https://www.adobe.com/content/dam/acom/en/corp...,PDF,8/1/2018,NaN,Explicit,Adobe is a powerhouse for SDGs. but not for 3 ...,U.S.
4,Adobe,13,Adobe Alignment to UN Sustainable Development ...,https://www.adobe.com/content/dam/acom/en/corp...,PDF,8/1/2018,NaN,Explicit,Adobe is a powerhouse for SDGs. but not for 3 ...,U.S.
5,ADP,1,Corporate Social Responsibility Report,https://www.adp.com/about-adp/corporate-social...,PDF,2/1/2018,NaN,Implicit,Very passionate about environmental sustainabi...,"Zambia, Uganda, Swaziland, Belize, El Salvador..."
6,ADP,2,Corporate Social Responsibility Report,https://www.adp.com/about-adp/corporate-social...,PDF,2/1/2018,NaN,Implicit,Very passionate about environmental sustainabi...,"Zambia, Uganda, Swaziland, Belize, El Salvador..."
7,ADP,3,Corporate Social Responsibility Report,https://www.adp.com/about-adp/corporate-social...,PDF,2/1/2018,NaN,Implicit,Very passionate about environmental sustainabi...,"Zambia, Uganda, Swaziland, Belize, El Salvador..."
8,ADP,4,Corporate Social Responsibility Report,https://www.adp.com/about-adp/corporate-social...,PDF,2/1/2018,NaN,Implicit,Very passionate about environmental sustainabi...,"Zambia, Uganda, Swaziland, Belize, El Salvador..."
9,ADP,5,Corporate Social Responsibility Report,https://www.adp.com/about-adp/corporate-social...,PDF,2/1/2018,NaN,Implicit,Very passionate about environmental sustainabi...,"Zambia, Uganda, Swaziland, Belize, El Salvador..."


In [6]:
#Making the country NaNs string "nan"s in order to pass through repRow
cleanTest.iloc[:, 9] = cleanTest.iloc[:, 9].fillna("nan")
np.unique(cleanTest.iloc[:, 9])

array(['Australia', 'Chile, Haiti, Japan, Pakistan, United States',
       'China, India, Israel, Japan, Malaysia, Philippines, Thailand, United States',
       'Germany, USA', 'Global',
       'Global, United States, United Kingdom, France, Australia, China, Hong Kong, Japan, Singapore, Netherlands, Germany, Sweden',
       'Globe', 'Globe, UK, United States',
       'Haiti, India, Mozambique, Peru, Sri Lanka, Tanzania',
       'Honduras, Bolivia, India; Malawai, Zambia', 'India',
       'Israel, United States, China', 'Lebanon', 'Local', 'Mexico',
       'Nigeria, Uganda, DRC, Afghanistan', 'Rwanda', 'South Africa',
       'U.S.', 'U.S., the United Kingdom', 'United States',
       'United States (local)',
       'United States, Africa, Central Asia, Middle East',
       'United States, Australia, India, Guatemala, Kenya, Madagascar, Borneo, Brazil,\nZimbabwe, and Sierra Leone',
       'United States, DRC', 'United States, Globe',
       'United States, Haiti', 'United States, North 

In [7]:
################################
## Cleanup of Country wording ##
################################

#Getting rid of newline characters
cleanTest.iloc[:, 9] = [re.sub('\\n', "", row) for row in cleanTest.iloc[:, 9]]

#Getting rid of the word "and"
cleanTest.iloc[:, 9] = [re.sub(' and ', "", row) for row in cleanTest.iloc[:, 9]]

#Getting rid of the word "the"
cleanTest.iloc[:, 9] = [re.sub(' the ', ",", row) for row in cleanTest.iloc[:, 9]]

#Changing all "U.S." to "United States"
cleanTest.iloc[:, 9] = [re.sub(' *U.?S.? *', "United States", row) for row in cleanTest.iloc[:, 9]] 

#Changing all "UK" to "United Kingdom"
cleanTest.iloc[:, 9] = [re.sub(' *U.?K.? *', "United States", row) for row in cleanTest.iloc[:, 9]] 

#Changing all "Globe" to "Global"
cleanTest.iloc[:, 9] = [re.sub(' *[Gg]lobe *', "Global", row) for row in cleanTest.iloc[:, 9]]


np.unique(cleanTest.iloc[:, 9])

array(['Australia', 'Chile, Haiti, Japan, Pakistan, United States',
       'China, India, Israel, Japan, Malaysia, Philippines, Thailand, United States',
       'Germany,United States', 'Global',
       'Global, United States, United Kingdom, France, Australia, China, Hong Kong, Japan, Singapore, Netherlands, Germany, Sweden',
       'Global,United StatesUnited States',
       'Haiti, India, Mozambique, Peru, Sri Lanka, Tanzania',
       'Honduras, Bolivia, India; Malawai, Zambia', 'India',
       'Israel, United States, China', 'Lebanon', 'Local', 'Mexico',
       'Nigeria, Uganda, DRC, Afghanistan', 'Rwanda', 'South Africa',
       'United States', 'United States (local)',
       'United States, Africa, Central Asia, Middle East',
       'United States, Australia, India, Guatemala, Kenya, Madagascar, Borneo, Brazil,Zimbabwe,Sierra Leone',
       'United States, DRC', 'United States, Haiti',
       'United States, North Korea', 'United States,,United Kingdom',
       'United States,Gl

In [8]:
len(cleanTest.iloc[:, 0])

309

In [9]:
cleanCountry = pd.DataFrame([])
for row in range(len(cleanTest.iloc[:, 0])):
    cleanCountry = cleanCountry.append(repRow(cleanTest.iloc[row, :], 9, 
                                        splitStr), 
                                 ignore_index = True)
cleanCountry

,Corporation,SDG(s),Source Name,Source,Source Type,Source Date,Source Author,Explicit or Implicit,Notes,Relevant Country
0,Adobe,4,Adobe Alignment to UN Sustainable Development ...,https://www.adobe.com/content/dam/acom/en/corp...,PDF,8/1/2018,NaN,Explicit,Adobe is a powerhouse for SDGs. but not for 3 ...,United States
1,Adobe,5,Adobe Alignment to UN Sustainable Development ...,https://www.adobe.com/content/dam/acom/en/corp...,PDF,8/1/2018,NaN,Explicit,Adobe is a powerhouse for SDGs. but not for 3 ...,United States
2,Adobe,8,Adobe Alignment to UN Sustainable Development ...,https://www.adobe.com/content/dam/acom/en/corp...,PDF,8/1/2018,NaN,Explicit,Adobe is a powerhouse for SDGs. but not for 3 ...,United States
3,Adobe,12,Adobe Alignment to UN Sustainable Development ...,https://www.adobe.com/content/dam/acom/en/corp...,PDF,8/1/2018,NaN,Explicit,Adobe is a powerhouse for SDGs. but not for 3 ...,United States
4,Adobe,13,Adobe Alignment to UN Sustainable Development ...,https://www.adobe.com/content/dam/acom/en/corp...,PDF,8/1/2018,NaN,Explicit,Adobe is a powerhouse for SDGs. but not for 3 ...,United States
5,ADP,1,Corporate Social Responsibility Report,https://www.adp.com/about-adp/corporate-social...,PDF,2/1/2018,NaN,Implicit,Very passionate about environmental sustainabi...,Zambia
6,ADP,1,Corporate Social Responsibility Report,https://www.adp.com/about-adp/corporate-social...,PDF,2/1/2018,NaN,Implicit,Very passionate about environmental sustainabi...,Uganda
7,ADP,1,Corporate Social Responsibility Report,https://www.adp.com/about-adp/corporate-social...,PDF,2/1/2018,NaN,Implicit,Very passionate about environmental sustainabi...,Swaziland
8,ADP,1,Corporate Social Responsibility Report,https://www.adp.com/about-adp/corporate-social...,PDF,2/1/2018,NaN,Implicit,Very passionate about environmental sustainabi...,Belize
9,ADP,1,Corporate Social Responsibility Report,https://www.adp.com/about-adp/corporate-social...,PDF,2/1/2018,NaN,Implicit,Very passionate about environmental sustainabi...,El Salvador


In [10]:
#Could not get rid of a space: doing it manually"
cleanCountry = cleanCountry[cleanCountry['Relevant Country'] != ""]

In [11]:
cleanCountry.to_csv("2019.02.21_Cleaned_Corporate_Donor_Sheet.csv")